In [ ]:
# default_exp api

# Omeka S API client

> Tools to interact with the Omeka S REST API

In [ ]:
#hide
from nbdev.showdoc import *
import random
import os
from copy import deepcopy
import pytest
%load_ext dotenv
%dotenv

In [ ]:
#export
import requests
import requests_cache
import json
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from pathlib import Path

class OmekaAPIClient(object):
    
    def __init__(self, api_url, key_identity=None, key_credential=None, use_cache=True):
        self.api_url = api_url
        self.params = {
            'key_identity': key_identity,
            'key_credential': key_credential
        }
        # Set up session and caching
        if use_cache:
            self.s = requests_cache.CachedSession(expire_after=3600)
            self.s.cache.clear()
        else:
            self.s = requests.Session()
        retries = Retry(total=10, backoff_factor=1, status_forcelist=[ 502, 503, 504, 524 ])
        self.s.mount('http://', HTTPAdapter(max_retries=retries))
        self.s.mount('https://', HTTPAdapter(max_retries=retries))
        
    def clear_cache():
        self.s.cache.clear()
        
    def process_response(self, response):
        '''
        Handle Omeka responses, raising exceptions on errors.
        '''
        # Raise exception on HTTP error
        response.raise_for_status()
        # Try extracting JSON data
        try:
            data = response.json()
        # If there's no JSON, display the raw response text and raise exception
        except (json.decoder.JSONDecodeError, ValueError):
            print(f'Bad JSON: {response.text}')
            raise
        else:
            return data
        
    def format_resource_id(self, resource_id, resource_type):
        '''
        Generate a formatted id for the resource with the specified Omeka id number and resource type.
        
        Parameters:
        * `resource_id` - numeric identifier used by Omeka for this resource
        * `resource_type` - one of Omeka's resource types, eg: 'items', 'properties'
        
        Returns:
        * a dict with values for '@id' and 'o:id'
        '''
        formatted_id = {
            '@id': f'{self.api_url}/{resource_type}/{resource_id}',
            'o:id': resource_id
        }
        return formatted_id

    def get_resources(self, resource_type, **kwargs):
        '''
        Get a list of resources matching the supplied parameters.
        This will return the first page of matching results. To retrieve additional pages, 
        you can supply the `page` parameter to move through the full result set.
        
        Parameters:
        * `resource_type` - one of Omeka's resource types, eg: 'items', 'properties'
        * there are many additional parameters you can supply as kwargs, see the Omeka documention
        
        Returns a dict with the following values:
        * `total_results` - number of matching resources
        * `results` - a list of dicts, each containing a JSON-LD formatted representation of a resource
        '''
        response = self.s.get(f'{self.api_url}/{resource_type}/', params=kwargs)
        data = self.process_response(response)
        return {'total_results': int(response.headers['Omeka-S-Total-Results']), 'results': data}
    
    def get_resource(self, resource_type, **kwargs):
        '''
        Get the first resource matching the supplied parameters.
        
        Parameters:
        * `resource_type` - one of Omeka's resource types, eg: 'items', 'properties'
        * there are many additional parameters you can supply as kwargs, see the Omeka documention
        
        Returns
        * a dict containing a JSON-LD formatted representation of the resource
        '''
        
        data = self.get_resources(resource_type, **kwargs)
        try:
            resource = data['results'][0]
        except IndexError:
            return
        else:
            return resource

    def get_resource_by_id(self, resource_id, resource_type='items'):
        '''
        Get a resource from its Omeka id.
        
        Parameters:
        * `resource_id` - numeric identifier used by Omeka for this resource
        * `resource_type` - one of Omeka's resource types, eg: 'items', 'properties'
        
        Returns
        * a dict containing a JSON-LD formatted representation of the resource
        '''
        response = self.s.get(f'{self.api_url}/{resource_type}/{resource_id}')
        data = self.process_response(response)
        return data

    def get_template_by_label(self, label):
        '''
        Get a resource template from its Omeka label.
        
        Parameters:
        * `label` - the name of the resource template in Omeka (eg. 'NewspaperArticle')
        
        Returns:
        * dict containing representation of the template
        '''
        return self.get_resource('resource_templates', label=label)
    
    def get_resource_by_term(self, term, resource_type='properties'):
        '''
        Get the resource (property or class) associated with the suppied term.
        
        Parameters:
        * `term` - property label qualified with vocabulary prefix (eg: 'schema:name')
        
        Returns:
        * dict containing representation of the resource
        '''
        return self.get_resource(resource_type, term=term)
    
    def get_resource_from_vocab(self, local_name, vocabulary_namespace_uri='http://schema.org/', resource_type='properties'):
        '''
        Get the resource (property or class) associated with the suppied vocabulary and label.
        
        Parameters:
        * `local_name` - label of the property or class
        * `vocabulary_namespace_uri` - URI defining the vocab
        
        Returns:
        * dict containing representation of the resource
        '''
        return self.get_resource(resource_type, local_name=local_name, vocabulary_namespace_uri=vocabulary_namespace_uri)
        
    def get_property_id(self, term):
        '''
        Get the numeric identifier associated with the supplied property term.
        
        Parameters:
        * `term` - property label qualified with vocabulary prefix (eg: 'schema:name')
        
        Returns:
        * numeric identifier
        '''
        resource = self.get_resource_by_term(term=term)
        if resource:
            return resource['o:id']

    def get_itemset_id(self, label):
        '''
        Get the numeric identifier associated with the supplied item-set label.
        
        Parameters:
        * `label` - item-set label as recorded in omeka (eg: 'my collection')
        
        Returns:
        * numeric identifier
        '''
        resource = self.get_resource('item_sets', search=label)
        if resource:
            return resource['o:id']



    def filter_items(self, params, **extra_filters):
        for filter_type in ['resource_template_id', 'resource_class_id', 'item_set_id', 'is_public']:
            filter_value = extra_filters.get(filter_type)
            if filter_value:
                params[filter_type] = filter_value
        return params
    
    def filter_items_by_property(self, filter_property='schema:name', filter_value='', filter_type='eq', page=1, **extra_filters):
        '''
        Filter the list of items by searching for a value in a particular property.
        Additional filters can also limit to items associated with particular templates, classes, or item sets.
        
        Parameters:
        * `filter_property` - property term (eg: 'schema:name')
        * `filter_value` - the value you want to find
        * `filter_type` - how `filter_value` should be compared to the stored values (eg: 'eq')
        * `page` - number of results page
        
        Additional parameters:
        * `resource_template_id` - numeric identifier
        * `resource_class_id` - numeric identifier
        * `item_set_id` - numeric identifier
        * `is_public` - boolean, True or False
        
        Returns a dict with the following values:
        * `total_results` - number of matching resources
        * `results` - a list of dicts, each containing a JSON-LD formatted representation of a resource
        
        '''
        # We need to get the id of the property we're using
        property_id = self.get_property_id(filter_property)
        params = {
            'property[0][joiner]': 'and', # and / or joins multiple property searches
            'property[0][property]': property_id, # property id
            'property[0][type]': filter_type, # See above for options
            'property[0][text]': filter_value,
            'page': page
        }
        params = self.filter_items(params, **extra_filters)
        # print(params)
        results = self.get_resources('items', **params)
        return results
    
    def search_items(self, query, search_type='fulltext_search', page=1, **extra_filters):
        '''
        Search for matching items.
        Two search types are available:
        * 'search` - looks for an exact match of the query in a property value
        * 'fulltext_search` - looks for the occurance of the query anywhere
        
        Parameters:
        * `query` - the text you want to search for
        * `search_type` - one of 'fulltext_search' or 'search'
        * `page` - number of results page
        
        Additional parameters:
        * `resource_template_id` - numeric identifier
        * `resource_class_id` - numeric identifier
        * `item_set_id` - numeric identifier
        * `is_public` - boolean, True or False
        
        Returns a dict with the following values:
        * `total_results` - number of matching resources
        * `results` - a list of dicts, each containing a JSON-LD formatted representation of a resource
        '''
        params = {'page': page}
        params[search_type] = query
        params = self.filter_items(params, **extra_filters)
        results = self.get_resources('items', **params)
        return results

    def get_template_properties(self, template_id):
        '''
        List properties used by the specified template.
        
        The resource template objects returned by the API don't include property terms.
        This function gets the additional details, and organises the properties in a dictionary, 
        organised by term. This makes it easy to check if a particular term is used by a template.
        
        Parameters:
        * `template_id` - numeric identifier for a template
        
        Returns:
        * a dict organised by property terms, with values for `property_id` and `type`
        '''
        properties = {}
        template = self.get_resource_by_id(template_id, 'resource_templates')
        for prop in template['o:resource_template_property']:
            prop_url = prop['o:property']['@id']
            # The resource template doesn't include property terms, so we have to go to the property data
            response = self.s.get(prop_url)
            data = self.process_response(response)
            # Use default data types if they're not defined in the resource template
            data_type = ['literal', 'uri', 'resource:item'] if prop['o:data_type'] == [] else prop['o:data_type']
            properties[data['o:term']] = {'property_id': data['o:id'], 'type': data_type}
        return properties
    
    # ADDING ITEMS
        
    def prepare_property_value(self, value, property_id):
        '''
        Formats a property value according to its datatype as expected by Omeka. 
        The formatted value can be used in a payload to create a new item.
        
        Parameters:
        * `value` - a dict containing a `value` and (optionally) a `type`
        * `property_id` - the numeric identifier of the property
        
        Note that is no `type` is supplied, 'literal' will be used by default.
        
        Returns:
        * a dict with values for `property_id`, `type`, and either `@id` or `@value`.
        '''
        if not isinstance(value, dict):
            value = {'value': value}
            
        try:
            data_type = value['type']
        except KeyError:
            data_type = 'literal'
        
        property_value = {
            'property_id': property_id,
            'type': data_type
        }
        
        if data_type == 'resource:item':
            property_value['@id'] = f'{self.api_url}/items/{value["value"]}'
            property_value['value_resource_id'] = value['value']
            property_value['value_resource_name'] = 'items'
        elif data_type == 'uri':
            property_value['@id'] = value['value']
        else:
            property_value['@value'] = value['value']
        return property_value

    def add_item(self, payload, media_files=None, template_id=None, class_id=None, item_set_id=None):
        '''
        Create a new item from the supplied payload, optionally uploading attached media files.
        
        Parameters:
        * `payload` - a dict generated by `prepare_item_payload()` or `prepare_item_payload_using_template()`
        * `media_files` - a list of paths pointing to media files, or a list of dicts with `path` and `title` values
        * `template_id` - internal Omeka identifier of a resource template you want to attach to this item
        * `class_id` - internal Omeka identifier of a resource class you want to attach to this item
        * `item_set_id` - internal Omeka identifier for an item set you want to add this item to
        
        Returns:
        * a dict providing the JSON-LD representation of the new item from Omeka
        '''
        if template_id:
            payload['o:resource_template'] = self.format_resource_id(template_id, 'resource_templates')
            # If class is not set explicitly, use class associated with template
            if not class_id:
                template = self.get_resource_by_id(template_id, 'resource_templates')
                class_id = template['o:resource_class']['o:id']
        if class_id:
            payload['o:resource_class'] = self.format_resource_id(class_id, 'resource_classes')
        if item_set_id:
            payload['o:item_set'] = self.format_resource_id(item_set_id, 'item_sets')
        if media_files:
            files = self.add_media_to_payload(payload, media_files)
            response = self.s.post(f'{self.api_url}/items', files=files, params=self.params)
        else:
            response = self.s.post(f'{self.api_url}/items', json=payload, params=self.params)
        #print(response.text)
        data = self.process_response(response)
        return data
    
    def prepare_item_payload(self, terms):
        '''
        Prepare an item payload, ready for upload.
        
        Parameters:
        * `terms`: a dict of terms, values, and (optionally) data types
        
        Returns:
        * the payload dict
        '''
        payload = {}
        for term, values in terms.items():
            # Get the property id of the supplied term
            try:
                property_id = self.get_property_id(term)
            except IndexError:
                print(f'Term "{term}" not found')
            else:
                payload[term] = []
                for value in values:
                    # Add a value formatted according to the data type
                    payload[term].append(self.prepare_property_value(value, property_id))
        return payload
    
    def prepare_item_payload_using_template(self, terms, template_id):
        '''
        Prepare an item payload, checking the supplied terms and values against the specified template.
        Note:
        * terms that are not in the template will generate a warning and be dropped from the payload
        * data types that don't match the template definitions will generate a warning and the term will be dropped from the payload
        * if no data type is supplied, a type that conforms with the template definition will be used
        
        Parameters:
        * `terms`: a dict of terms, values, and (optionally) data types
        * `template_id`: Omeka's internal numeric identifier for the template
        
        Returns:
        * the payload dict
        '''
        template_properties = self.get_template_properties(template_id)
        payload = {}
        for term, values in terms.items():
            if term in template_properties:
                property_details = template_properties[term]
                payload[term] = []
                for value in values:
                    if not isinstance(value, dict):
                        value = {'value': value}
                    # The supplied data type doesn't match the template
                    if 'type' in value and value['type'] not in property_details['type']:
                        print(f'Data type "{value["type"]}" for term "{term}" not allowed by template')
                        break
                    elif 'type' not in value:
                        # Use default datatype from template if none is supplied
                        if len(property_details['type']) == 1:
                            value['type'] = property_details['type'][0]
                        # Use literal if allowed by template and data type not supplied
                        elif 'literal' in property_details['type']:
                            value['type'] = 'literal'
                        # Don't know what data type to use
                        else:
                            print(f'Specify data type for term "{term}"')
                            break
                    # Add a value formatted according to the data type
                    payload[term].append(self.prepare_property_value(value, property_details['property_id']))
            # The supplied term is not in the template
            else:
                print(f'Term {term} not in template') 
        return payload
    
    def add_media_to_payload(self, payload, media_files):
        '''
        Add media files to the item payload.
        
        Parameters:
        * `payload` - the payload dict to be modified
        * `media_files` - media files to be uploaded
        
        The value of `media_files` can be either:
        * a list of paths to the image/media files (filename is used as title)
        * a list of dicts, each containing `title`, and `path` values
        
        The path values can either be strings or pathlib Paths.
        
        Returns: 
        * the modified payload dict
        '''
        payload['o:media'] = []
        files = {}
        for index, media_file in enumerate(media_files):
            if isinstance(media_file, dict):
                title = media_file['title']
                path = Path(media_file['path'])
            else:
                path = Path(media_file)
                title = path.name
            payload['o:media'].append({'o:ingester': 'upload', 'file_index': str(index), 'o:item': {}, 'dcterms:title': [{'property_id': 1, '@value': title, 'type': 'literal'}]})
            files[f'file[{index}]'] = path.read_bytes()
        files['data'] = (None, json.dumps(payload), 'application/json')
        #files['data'] = (json.dumps(payload), 'application/json')
        return files
    
    # UPDATING RESOURCES
    
    def delete_resource(self, resource_id, resource_type):
        '''
        Deletes a resource. No confirmation is requested, so use carefully.
        
        Parameters:
        * `resource_id` - local Omeka identifier of the resource you want to delete
        * `resource_type` - type of the resource (eg 'items')
        
        Returns:
        * dict with JSON-LD representation of the deleted resource
        '''
        response = self.s.delete(f'{self.api_url}/{resource_type}/{resource_id}', params=self.params)
        data = self.process_response(response)
        return data
    
    def update_resource(self, payload, resource_type='items'):
        '''
        Update an existing resource.
        
        Parameters:
        * `payload` - the updated resource data
        * `resource_type` - the type of resource
        
        To avoid problems, it's generally easiest to retrieve the resource first,
        make your desired changes to it, then submit the updated resource as your payload.
        '''
        response = self.s.put(f'{self.api_url}/{resource_type}/{payload["o:id"]}', json=payload, params=self.params)
        data = self.process_response(response)
        return data
    
    def add_media_to_item(self, item_id, media_file, payload={}, template_id=None, class_id=None):
        '''
        Upload a media file and associate it with an existing item.
        
        Parameters:
        * `item_id` - the Omeka id of the item this media file should be added to
        * `media_path` - a path to an image/media file (string or pathlib Path)
        * `payload` (optional) - metadata to attach to media object, either 
           a dict generated by `prepare_item_payload()` or `prepare_item_payload_using_template()`,
           or a string which is used as the value for `dcterms:title`.
        * `template_id` - internal Omeka identifier of a resource template you want to attach to this item
        * `class_id` - internal Omeka identifier of a resource class you want to attach to this item
        
        Returns: 
        * a dict providing a JSON-LD representation of the new media object
        '''
        files = {}
        # For backwards compatibility
        if isinstance(media_file, dict):
            path = media_file['path']
            payload = media_file['title']
        # Make sure path is a Path object
        path = Path(media_file)
        if isinstance(payload, str):
            payload = self.prepare_item_payload({'dcterms:title': [payload]})
        if template_id:
            payload['o:resource_template'] = self.format_resource_id(template_id, 'resource_templates')
            if not class_id:
                template = self.get_resource_by_id(template_id, 'resource_templates')
                class_id = template['o:resource_class']['o:id']
        if class_id:
            payload['o:resource_class'] = self.format_resource_id(class_id, 'resource_classes')
        file_data = {
            'o:ingester': 'upload', 
            'file_index': '0',
            'o:source': path.name,
            'o:item': {'o:id': item_id},
        }
        payload.update(file_data)
        files[f'file[0]'] = path.read_bytes()
        files['data'] = (None, json.dumps(payload), 'application/json')
        response = self.s.post(f'{self.api_url}/media', files=files, params=self.params)
        data = self.process_response(response)
        return data
    
    # MANAGING TEMPLATES
    
    def localise_custom_vocabs(self, data_types):
        '''
        Check a list of data types for references to custom vocabs.
        If found, look for the local identifier of the custom vocab, 
        and insert it into the data type information.
        
        Parameters:
        * `data_types` - a list of data types from an exported template property
        
        Returns:
        * list of datatypes with local identifiers
        '''
        dt_names = []
        for dt in data_types:
            if dt['name'].startswith('customvocab'):
                try:
                    cv_id = self.get_resource('custom_vocabs', label=dt['label'])['o:id']
                except TypeError:
                    print(f'Custom vocab {dt["label"]} not found')
                else:
                    dt_names.append(f'customvocab:{cv_id}')
            else:
                dt_names.append(dt['name'])
        return dt_names
    
    def get_template_class_id(self, template):
        '''
        Get the local id of the resource class associated with the supplied template.
        
        Parameters:
        * `template` - dict from exported template
        
        Returns:
        * class identifier
        '''
        resource_class = self.get_resource_from_vocab(
            local_name=template['o:resource_class']['local_name'],
            vocabulary_namespace_uri=template['o:resource_class']['vocabulary_namespace_uri'],
            resource_type='resource_classes'
        )
        if resource_class:
            return resource_class['o:id']
        else:
            print(f'Resource class "{template["o:resource_class"]["local_name"]}" not found')
    
    def get_template_property_id(self, template, term):
        '''
        Get the local id of the property associated with the supplied template.
        
        Parameters:
        * `template` - dict from exported template
        * `term` - property term (eg 'o:title_property')
        
        Returns:
        * property identifier
        '''
        prop = self.get_resource_from_vocab(
            local_name=template[term]['local_name'],
            vocabulary_namespace_uri=template[term]['vocabulary_namespace_uri'],
            resource_type='properties'
        )
        if prop:
            return prop['o:id']
        else:
            print(f'Property "{template[term]["local_name"]}" not found')
    
    def prepare_template_payload(self, template_file):
        '''
        Insert local property, class, and vocab identifiers into a resource template
        exported from Omeka so that it can be uploaded to the local instance.
        
        Parameters:
        * `template_file` - path to a template exported from Omeka (str or pathlib Path)
        
        Returns:
        * template payload with local identifiers inserted
        '''
        # Load the template file from the filesystem
        template = json.loads(Path(template_file).read_bytes())
        # Get local resource class id
        resource_class_id = self.get_template_class_id(template)
        # Get id of property used for title
        title_id = self.get_template_property_id(template, 'o:title_property')
        # Get id of property used for description
        description_id = self.get_template_property_id(template, 'o:description_property')
        # Create skeleton payload
        template_payload = {
            'o:label': template['o:label'],
            'o:resource_class': self.format_resource_id(resource_class_id, 'resource_classes'),
            'o:title_property': self.format_resource_id(title_id, 'properties'),
            'o:description_property': self.format_resource_id(description_id, 'properties'),
            'o:resource_template_property': []
        }
        # The property list in the JSON file exported from Omeka doesn't include property ids, so we need to add them.
        for prop in template['o:resource_template_property']:
            # Keep the namespaced values in the property dictionary
            prop_payload = {k: v for k, v in prop.items() if k.startswith('o:')}

            # Add data types
            prop_payload['o:data_type'] = self.localise_custom_vocabs(prop['data_types'])

            # Get the property id
            prop_data = self.get_resource_from_vocab(
                local_name=prop['local_name'],
                vocabulary_namespace_uri=prop['vocabulary_namespace_uri'],
                resource_type='properties'
            )
            if prop_data:

                # Add property id to payload
                prop_payload['o:property'] = self.format_resource_id(prop_data['o:id'], 'properties')

                # Add the property to the template
                template_payload['o:resource_template_property'].append(prop_payload)
            else:
                print(f'Property "{prop["label"]}" not found')
  
        return template_payload

    def upload_template(self, template_payload):
        '''
        Upload a template exported from an instance of Omeka to the current local instance.
        
        Parameters:
        * `template_payload` - dict payload generated by `prepare_template_payload`
        
        Return:
        * dict containing a JSON-LD representation of the uploaded template
        '''
        # Upload the template payload
        response = self.s.post(f'{self.api_url}/resource_templates/', params=self.params, json=template_payload)
        data = self.process_response(response)
        return data
        
    # MODULE RELATED METHODS

    def add_marker_to_item(self, item_id, coords=None, terms=None, label=None, media_id=None):
        '''
        Add a map marker to an item.
        Requires the `mapping` module to be installed.
        
        Parameters:
        * `item_id` - identifier of item to add marker to
        * `coords` - list with coordinates in longitude, latitude order eg [151.209900, -33.865143]
        * `terms` - list with vocab terms containing longitude and latitude values eg ['schema:longitude', 'schema:latitude']
        * `label` - label for marker (defaults to item title)
        * `media_id` - identifier of media resource to display with marker
        
        Returns:
        * dict providing JSON-LD representation of marker
        '''
        item = self.get_resource_by_id(item_id)
        if coords:
            lon, lat = coords
        elif terms:
            lon, lat = terms
            lon = item[lon][0]['@value']
            lat = item[lat][0]['@value']
        else:
            lon = item['schema:longitude'][0]['@value']
            lat = item['schema:latitude'][0]['@value']
        lon = float(lon)
        lat = float(lat)
        if not label:
            label = item['o:title']
        marker_payload = {
            'o:item': {'o:id': item_id},
            'o-module-mapping:lat': lat,
            'o-module-mapping:lng': lon,
            'o-module-mapping:label': label
        }
        if media_id:
            marker_payload['o:media'] = {'o:id': media_id}
        response = self.s.post(f'{self.api_url}/mapping_markers/', json=marker_payload, params=self.params)
        data = self.process_response(response)
        return data


First import the `OmekaAPIClient` class.

``` python
from omeka_s_tools.api import OmekaAPIClient
```

To initialise the `OmekaAPIClient` you need to provide the base url of your Omeka instance's API.

In [ ]:
omeka = OmekaAPIClient('http://timsherratt.org/collections/api')

This will let you access details of all public resources provided by your Omeka instance. To access private resources, or create new resources, you'll have to authenticate your API requests by provide a `key_identity` and `key_credential`. See [Adding items](#Adding-items) for an example.

## Getting resources

You can find a [full list of Omeka's resources](https://omeka.org/s/docs/developer/api/#resources) in the API documentation. You're most likely to be interested in 'items', but accessing 'properties' and 'resource_templates' can be useful when you're creating new resources.

You should be able to use `get_resources` with any of the resource types, though some of the parameters will be different. 

In [ ]:
show_doc(OmekaAPIClient.get_resources)

<h4 id="OmekaAPIClient.get_resources" class="doc_header"><code>OmekaAPIClient.get_resources</code><a href="__main__.py#L63" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.get_resources</code>(**`resource_type`**, **\*\*`kwargs`**)

Get a list of resources matching the supplied parameters.
This will return the first page of matching results. To retrieve additional pages, 
you can supply the `page` parameter to move through the full result set.

Parameters:
* `resource_type` - one of Omeka's resource types, eg: 'items', 'properties'
* there are many additional parameters you can supply as kwargs, see the Omeka documention

Returns a dict with the following values:
* `total_results` - number of matching resources
* `results` - a list of dicts, each containing a JSON-LD formatted representation of a resource

In [ ]:
data = omeka.get_resources('items')
data['total_results']

120

In [ ]:
assert isinstance(data['total_results'], int)

In [ ]:
# Get a list of available resource templates
data = omeka.get_resources('resource_templates')
first_template = data['results'][0]
assert first_template['@type'] == 'o:ResourceTemplate'

In [ ]:
show_doc(OmekaAPIClient.get_resource_by_id)

<h4 id="OmekaAPIClient.get_resource_by_id" class="doc_header"><code>OmekaAPIClient.get_resource_by_id</code><a href="__main__.py#L101" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.get_resource_by_id</code>(**`resource_id`**, **`resource_type`**=*`'items'`*)

Get a resource from its Omeka id.

Parameters:
* `resource_id` - numeric identifier used by Omeka for this resource
* `resource_type` - one of Omeka's resource types, eg: 'items', 'properties'

Returns
* a dict containing a JSON-LD formatted representation of the resource

In [ ]:
# Get a random item from a list of items
data = omeka.get_resources('items')
random_item = random.choice(data['results'])
item_id = random_item['o:id']

# Get the same item using its id
item_from_id = omeka.get_resource_by_id(item_id, 'items')

# Check that they're the same
assert random_item == item_from_id

It can be difficult to remember all the available parameters and the differences between resource types, so I've created a number of more specialised methods that are really just wrappers around `get_resources`.

In [ ]:
show_doc(OmekaAPIClient.get_template_by_label)

<h4 id="OmekaAPIClient.get_template_by_label" class="doc_header"><code>OmekaAPIClient.get_template_by_label</code><a href="__main__.py#L116" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.get_template_by_label</code>(**`label`**)

Get a resource template from its Omeka label.

Parameters:
* `label` - the name of the resource template in Omeka (eg. 'NewspaperArticle')

Returns:
* dict containing representation of the template

In [ ]:
newspaper_template = omeka.get_template_by_label('Newspaper')

assert newspaper_template['@type'] == 'o:ResourceTemplate'
assert newspaper_template['o:label'] == 'Newspaper'

In [ ]:
# Get a random template from list
template_from_list = random.choice(omeka.get_resources('resource_templates')['results'])
label = template_from_list['o:label']

# Get the template using label
template_from_label = omeka.get_template_by_label(label)

# Check they're both the same
assert template_from_list == template_from_label

In [ ]:
show_doc(OmekaAPIClient.get_resource_by_term)

<h4 id="OmekaAPIClient.get_resource_by_term" class="doc_header"><code>OmekaAPIClient.get_resource_by_term</code><a href="__main__.py#L128" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.get_resource_by_term</code>(**`term`**, **`resource_type`**=*`'properties'`*)

Get the resource (property or class) associated with the suppied term.

Parameters:
* `term` - property label qualified with vocabulary prefix (eg: 'schema:name')

Returns:
* dict containing representation of the resource

In [ ]:
prop = omeka.get_resource_by_term('schema:name')
prop

{'@context': 'http://timsherratt.org/collections/api-context',
 '@id': 'http://timsherratt.org/collections/api/properties/1116',
 '@type': 'o:Property',
 'o:id': 1116,
 'o:local_name': 'name',
 'o:label': 'name',
 'o:comment': 'The name of the item.',
 'o:term': 'schema:name',
 'o:vocabulary': {'@id': 'http://timsherratt.org/collections/api/vocabularies/5',
  'o:id': 5}}

In [ ]:
assert prop['@type'] == 'o:Property'
assert prop['o:term'] == 'schema:name'

In [ ]:
show_doc(OmekaAPIClient.get_resource_from_vocab)

<h4 id="OmekaAPIClient.get_resource_from_vocab" class="doc_header"><code>OmekaAPIClient.get_resource_from_vocab</code><a href="__main__.py#L140" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.get_resource_from_vocab</code>(**`local_name`**, **`vocabulary_namespace_uri`**=*`'http://schema.org/'`*, **`resource_type`**=*`'properties'`*)

Get the resource (property or class) associated with the suppied vocabulary and label.

Parameters:
* `local_name` - label of the property or class
* `vocabulary_namespace_uri` - URI defining the vocab

Returns:
* dict containing representation of the resource

In [ ]:
prop = omeka.get_resource_from_vocab(
    'name', 
    vocabulary_namespace_uri='http://schema.org/',
    resource_type='properties')
prop

{'@context': 'http://timsherratt.org/collections/api-context',
 '@id': 'http://timsherratt.org/collections/api/properties/1116',
 '@type': 'o:Property',
 'o:id': 1116,
 'o:local_name': 'name',
 'o:label': 'name',
 'o:comment': 'The name of the item.',
 'o:term': 'schema:name',
 'o:vocabulary': {'@id': 'http://timsherratt.org/collections/api/vocabularies/5',
  'o:id': 5}}

In [ ]:
vocab = requests.get(prop['o:vocabulary']['@id'])
namespace_uri = vocab.json()['o:namespace_uri']

assert namespace_uri == 'http://schema.org/'
assert prop['o:local_name'] == 'name'

In [ ]:
show_doc(OmekaAPIClient.get_property_id)

<h4 id="OmekaAPIClient.get_property_id" class="doc_header"><code>OmekaAPIClient.get_property_id</code><a href="__main__.py#L153" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.get_property_id</code>(**`term`**)

Get the numeric identifier associated with the supplied property term.

Parameters:
* `term` - property label qualified with vocabulary prefix (eg: 'schema:name')

Returns:
* numeric identifier

In [ ]:
prop_id = omeka.get_property_id('schema:name')
prop_id

1116

In [ ]:
assert isinstance(prop_id, int)

prop = omeka.get_resource_by_id(prop_id, 'properties')
assert prop['o:term'] == 'schema:name'

In [ ]:
show_doc(OmekaAPIClient.filter_items_by_property)

<h4 id="OmekaAPIClient.filter_items_by_property" class="doc_header"><code>OmekaAPIClient.filter_items_by_property</code><a href="__main__.py#L174" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.filter_items_by_property</code>(**`filter_property`**=*`'schema:name'`*, **`filter_value`**=*`''`*, **`filter_type`**=*`'eq'`*, **`page`**=*`1`*, **\*\*`extra_filters`**)

Filter the list of items by searching for a value in a particular property.
Additional filters can also limit to items associated with particular templates, classes, or item sets.

Parameters:
* `filter_property` - property term (eg: 'schema:name')
* `filter_value` - the value you want to find
* `filter_type` - how `filter_value` should be compared to the stored values (eg: 'eq')
* `page` - number of results page

Additional parameters:
* `resource_template_id` - numeric identifier
* `resource_class_id` - numeric identifier
* `item_set_id` - numeric identifier
* `is_public` - boolean, True or False

Returns a dict with the following values:
* `total_results` - number of matching resources
* `results` - a list of dicts, each containing a JSON-LD formatted representation of a resource

In [ ]:
items = omeka.filter_items_by_property(filter_property='schema:name', filter_value='wragge', filter_type='in')

assert len(items['results']) > 0
assert 'wragge' in items['results'][0]['schema:name'][0]['@value'].lower()

In [ ]:
items['results'][0]['schema:name'][0]['@value']

'Clement Wragge at Home. - UPDATED! - UPDATED!'

In [ ]:
show_doc(OmekaAPIClient.search_items)

<h4 id="OmekaAPIClient.search_items" class="doc_header"><code>OmekaAPIClient.search_items</code><a href="__main__.py#L210" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.search_items</code>(**`query`**, **`search_type`**=*`'fulltext_search'`*, **`page`**=*`1`*, **\*\*`extra_filters`**)

Search for matching items.
Two search types are available:
* 'search` - looks for an exact match of the query in a property value
* 'fulltext_search` - looks for the occurance of the query anywhere

Parameters:
* `query` - the text you want to search for
* `search_type` - one of 'fulltext_search' or 'search'
* `page` - number of results page

Additional parameters:
* `resource_template_id` - numeric identifier
* `resource_class_id` - numeric identifier
* `item_set_id` - numeric identifier
* `is_public` - boolean, True or False

Returns a dict with the following values:
* `total_results` - number of matching resources
* `results` - a list of dicts, each containing a JSON-LD formatted representation of a resource

In [ ]:
items = omeka.search_items('wragge')
items['total_results']

23

In [ ]:
assert items['total_results'] >= len(items['results'])

## Adding items

To add resources to Omeka you need authenticate your API requests by supplying `key_identity` and `key_credential` tokens as parameters.

In [ ]:
omeka_auth = OmekaAPIClient(
    api_url='https://timsherratt.org/collections/api',
    key_identity=os.getenv('KEY_IDENTITY'), 
    key_credential=os.getenv('KEY_CREDENTIAL')
)

There are two stages in creating a new item:

* prepare a payload containing the item data in the format expected by Omeka
* upload the payload to Omeka

### Prepare an item payload

In [ ]:
show_doc(OmekaAPIClient.prepare_item_payload)

<h4 id="OmekaAPIClient.prepare_item_payload" class="doc_header"><code>OmekaAPIClient.prepare_item_payload</code><a href="__main__.py#L336" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.prepare_item_payload</code>(**`terms`**)

Prepare an item payload, ready for upload.

Parameters:
* `terms`: a dict of terms, values, and (optionally) data types

Returns:
* the payload dict

In [ ]:
test_item = {
    'dcterms:title': [
        {
            'value': 'My first resource!'
        }
    ]
}

In [ ]:
payload = omeka_auth.prepare_item_payload(test_item)
payload

{'dcterms:title': [{'property_id': 1,
   'type': 'literal',
   '@value': 'My first resource!'}]}

Each term in the payload should have values for `property_id` and `type`, and the property value should be included as either `@id` or `@value` depending on the data type.

In [ ]:
first_property = list(payload.values())[0][0]

assert 'property_id' in first_property
assert 'type' in first_property
assert '@value' in first_property or '@id' in first_property

One of the powerful features of Omeka S is the ability to create resource templates that define a set of properties for a particular type of item. We can use templates to help create our payload, ensuring that the terms and data types we supply match those expected by the template. This is useful for identifying potential problems before we upload to Omeka, especially as Omeka will sometimes just fail silently.

In [ ]:
show_doc(OmekaAPIClient.prepare_item_payload_using_template)

<h4 id="OmekaAPIClient.prepare_item_payload_using_template" class="doc_header"><code>OmekaAPIClient.prepare_item_payload_using_template</code><a href="__main__.py#L360" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.prepare_item_payload_using_template</code>(**`terms`**, **`template_id`**)

Prepare an item payload, checking the supplied terms and values against the specified template.
Note:
* terms that are not in the template will generate a warning and be dropped from the payload
* data types that don't match the template definitions will generate a warning and the term will be dropped from the payload
* if no data type is supplied, a type that conforms with the template definition will be used

Parameters:
* `terms`: a dict of terms, values, and (optionally) data types
* `template_id`: Omeka's internal numeric identifier for the template

Returns:
* the payload dict

Once you've created a payload, you can add paths to any media files that you want to be attached to the item.

In [ ]:
test_newspaper_item = {
    'schema:name': [
        {
            'value': 'Wragge!'
        }
    ]
}

In [ ]:
test_newspaper_payload = omeka_auth.prepare_item_payload_using_template(test_newspaper_item, 5)
test_newspaper_payload

{'schema:name': [{'property_id': 1116,
   'type': 'literal',
   '@value': 'Wragge!'}]}

In [ ]:
assert test_newspaper_item['schema:name'][0]['value'] == test_newspaper_payload['schema:name'][0]['@value']
assert test_newspaper_payload['schema:name'][0]['type'] == 'literal'

In [ ]:
show_doc(OmekaAPIClient.add_media_to_payload)

<h4 id="OmekaAPIClient.add_media_to_payload" class="doc_header"><code>OmekaAPIClient.add_media_to_payload</code><a href="__main__.py#L406" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.add_media_to_payload</code>(**`payload`**, **`media_files`**)

Add media files to the item payload.

Parameters:
* `payload` - the payload dict to be modified
* `media_files` - media files to be uploaded

The value of `media_files` can be either:
* a list of paths to the image/media files (filename is used as title)
* a list of dicts, each containing `title`, and `path` values

The path values can either be strings or pathlib Paths.

Returns: 
* the modified payload dict

In [ ]:
payload_with_media = omeka_auth.add_media_to_payload({}, media_files=['media/nla.news-article226799674-24144902.jpg'])

In [ ]:
assert json.loads(payload_with_media['data'][1])

### Upload the payload to Omeka and create a new item

In [ ]:
show_doc(OmekaAPIClient.add_item)

<h4 id="OmekaAPIClient.add_item" class="doc_header"><code>OmekaAPIClient.add_item</code><a href="__main__.py#L303" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.add_item</code>(**`payload`**, **`media_files`**=*`None`*, **`template_id`**=*`None`*, **`class_id`**=*`None`*, **`item_set_id`**=*`None`*)

Create a new item from the supplied payload, optionally uploading attached media files.

Parameters:
* `payload` - a dict generated by `prepare_item_payload()` or `prepare_item_payload_using_template()`
* `media_files` - a list of paths pointing to media files, or a list of dicts with `path` and `title` values
* `template_id` - internal Omeka identifier of a resource template you want to attach to this item
* `class_id` - internal Omeka identifier of a resource class you want to attach to this item
* `item_set_id` - internal Omeka identifier for an item set you want to add this item to

Returns:
* a dict providing the JSON-LD representation of the new item from Omeka

### Add a simple item

In [ ]:
new_item = omeka_auth.add_item(payload)
new_item

{'@context': 'https://timsherratt.org/collections/api-context',
 '@id': 'https://timsherratt.org/collections/api/items/1038',
 '@type': 'o:Item',
 'o:id': 1038,
 'o:is_public': True,
 'o:owner': {'@id': 'https://timsherratt.org/collections/api/users/1',
  'o:id': 1},
 'o:resource_class': None,
 'o:resource_template': None,
 'o:thumbnail': None,
 'o:title': 'My first resource!',
 'thumbnail_display_urls': {'large': None, 'medium': None, 'square': None},
 'o:created': {'@value': '2022-04-04T00:47:02+00:00',
  '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
 'o:modified': {'@value': '2022-04-04T00:47:02+00:00',
  '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
 'o:media': [],
 'o:item_set': [],
 'o:site': [],
 'dcterms:title': [{'type': 'literal',
   'property_id': 1,
   'property_label': 'Title',
   'is_public': True,
   '@value': 'My first resource!'}]}

In [ ]:
assert new_item['@type'] == 'o:Item'
assert test_item['dcterms:title'][0]['value'] == new_item['dcterms:title'][0]['@value']

You can also add an item to an existing item set by supplying the `item_set_id` parameter.

In [ ]:
# Get a random item set id
random_item_set = random.choice(omeka_auth.get_resources('item_sets')['results'])
item_set_id = random_item_set['o:id']

# Create a new item using the item set id
new_item_in_item_set = new_item = omeka_auth.add_item(payload, item_set_id=item_set_id)

# Check that the ids match
assert new_item_in_item_set['o:item_set'][0]['o:id'] == item_set_id

## Updating resources

In [ ]:
show_doc(OmekaAPIClient.delete_resource)

<h4 id="OmekaAPIClient.delete_resource" class="doc_header"><code>OmekaAPIClient.delete_resource</code><a href="__main__.py#L440" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.delete_resource</code>(**`resource_id`**, **`resource_type`**)

Deletes a resource. No confirmation is requested, so use carefully.

Parameters:
* `resource_id` - local Omeka identifier of the resource you want to delete
* `resource_type` - type of the resource (eg 'items')

Returns:
* dict with JSON-LD representation of the deleted resource

In [ ]:
# Get a random item from a list of items
data = omeka.get_resources('items')
random_item = random.choice(data['results'])

deleted_resource = omeka_auth.delete_resource(random_item['o:id'], 'items')

# The id of the deleted item should be None
assert deleted_resource['o:id'] is None

# The titles of the random and deleted items should be the same
assert deleted_resource['o:title'] == random_item['o:title']

# Trying to get the random item should now raise an HTTP 404 error
with pytest.raises(requests.exceptions.HTTPError):
    omeka_auth.get_resource_by_id(random_item['o:id'], 'items')

In [ ]:
show_doc(OmekaAPIClient.update_resource)

<h4 id="OmekaAPIClient.update_resource" class="doc_header"><code>OmekaAPIClient.update_resource</code><a href="__main__.py#L455" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.update_resource</code>(**`payload`**, **`resource_type`**=*`'items'`*)

Update an existing resource.

Parameters:
* `payload` - the updated resource data
* `resource_type` - the type of resource

To avoid problems, it's generally easiest to retrieve the resource first,
make your desired changes to it, then submit the updated resource as your payload.

In [ ]:
# Get a random item from a list of items
data = omeka_auth.filter_items_by_property(filter_property='schema:name', filter_type='ex')
random_item = random.choice(data['results'])
# Original title
title = random_item['schema:name'][0]['@value']

# Copy and update the original item
new_item = deepcopy(random_item)
new_item['schema:name'][0]['@value'] = title + ' - UPDATED!'

# Update the item 
updated_item = omeka_auth.update_resource(new_item, 'items')

# Display the updated title
updated_title = updated_item['schema:name'][0]['@value']
print(updated_title)

# The id of the original and upated items should be the same
assert random_item['o:id'] == updated_item['o:id']

# But the titles should be different
assert title != updated_item['schema:name'][0]['@value']

Wellington Times (NSW : 1899 - 1954) - UPDATED! - UPDATED! - UPDATED!


In [ ]:
show_doc(OmekaAPIClient.add_media_to_item)

<h4 id="OmekaAPIClient.add_media_to_item" class="doc_header"><code>OmekaAPIClient.add_media_to_item</code><a href="__main__.py#L470" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.add_media_to_item</code>(**`item_id`**, **`media_file`**, **`payload`**=*`{}`*, **`template_id`**=*`None`*, **`class_id`**=*`None`*)

Upload a media file and associate it with an existing item.

Parameters:
* `item_id` - the Omeka id of the item this media file should be added to
* `media_path` - a path to an image/media file (string or pathlib Path)
* `payload` (optional) - metadata to attach to media object, either 
   a dict generated by `prepare_item_payload()` or `prepare_item_payload_using_template()`,
   or a string which is used as the value for `dcterms:title`.
* `template_id` - internal Omeka identifier of a resource template you want to attach to this item
* `class_id` - internal Omeka identifier of a resource class you want to attach to this item

Returns: 
* a dict providing a JSON-LD representation of the new media object

In [ ]:
omeka_auth = OmekaAPIClient(
    api_url='https://timsherratt.org/collections/api',
    key_identity=os.getenv('KEY_IDENTITY'), 
    key_credential=os.getenv('KEY_CREDENTIAL')
)

Add a media file to an item without any additional metadata.

In [ ]:
# Get a random item from a list of items
data = omeka_auth.get_resources('items')
item = random.choice(data['results'])
item_id = item['o:id']

# Add a new media file to the item
media_resource = omeka_auth.add_media_to_item(item_id, 'media/nla.news-article226799674-24144902.jpg')

# Check that the media resource is the expected type
assert media_resource['@type'] == 'o:Media'

# Check that the media resource is linked to the item
assert media_resource['o:item']['o:id'] == item_id

Include a title for a new media item by supplying a string value to the `payload` parameter.

In [ ]:
# Add a new media file with a title to the item
media_resource = omeka_auth.add_media_to_item(item_id, 'media/nla.news-article226799674-24144902.jpg', payload='This has a title')

# The title should be saved as dcterms:title
assert media_resource['dcterms:title'][0]['@value'] == 'This has a title'

Include more detailed metadata with a new media item by generating a payload and supplying it using the `payload` parameter.

In [ ]:
# Define some metadata
media_metadata = {
    'schema:name': ['This is a different title'],
    'schema:identifier': ['myidis3']
}

# Convert metadata into an Omeka payload
media_payload = omeka_auth.prepare_item_payload(media_metadata)

# Add a new media file with a metadata payload to the item
media_resource = omeka_auth.add_media_to_item(item_id, 'media/nla.news-article226799674-24144902.jpg', payload=media_payload)

assert media_resource['schema:name'][0]['@value'] == 'This is a different title'

Use a specific resource template with the new media item by using the `template_id` parameter.

In [ ]:
# Get a random template from a list of available templates for testing
data = omeka_auth.get_resources('resource_templates')
template = random.choice(data['results'])
template_id = template['o:id']

# Add a new media file and assign it to a template using the `template_id` parameter
media_resource = omeka_auth.add_media_to_item(item_id, 'media/nla.news-article226799674-24144902.jpg', payload=media_payload, template_id=template_id)

# Check the new media object has the correct template id
assert media_resource['o:resource_template']['o:id'] == template_id

In [ ]:
show_doc(OmekaAPIClient.add_marker_to_item)

<h4 id="OmekaAPIClient.add_marker_to_item" class="doc_header"><code>OmekaAPIClient.add_marker_to_item</code><a href="__main__.py#L653" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.add_marker_to_item</code>(**`item_id`**, **`coords`**=*`None`*, **`terms`**=*`None`*, **`label`**=*`None`*, **`media_id`**=*`None`*)

Add a map marker to an item.
Requires the `mapping` module to be installed.

Parameters:
* `item_id` - identifier of item to add marker to
* `coords` - list with coordinates in longitude, latitude order eg [151.209900, -33.865143]
* `terms` - list with vocab terms containing longitude and latitude values eg ['schema:longitude', 'schema:latitude']
* `label` - label for marker (defaults to item title)
* `media_id` - identifier of media resource to display with marker

Returns:
* dict providing JSON-LD representation of marker

The Omeka [Mapping](https://omeka.org/s/modules/Mapping/) module lets you associate an item with a location on a map. Once you have the module installed, you can use the Omeka API to add map markers to items.

In [ ]:
# Get a random item from a list of items
data = omeka_auth.get_resources('items')
item = random.choice(data['results'])
item_id = item['o:id']

# Add a marker using coords
marker = omeka_auth.add_marker_to_item(item_id, coords=[151.209900, -33.865143])

assert marker['@type'] == 'o-module-mapping:Marker'
assert marker['o:item']['o:id'] == item_id

You can also read the map coordinates from the item itself. By default `OmekaAPIClient.add_marker_to_item` will look for values in `schema:longitude` and `schema:latitude`, but you can specify the vocabulary terms that you want to look for.

Here we'll create a simple item with values for `schema:longitude` and `schema:latitude`.

In [ ]:
test_place = {
    'schema:name': [
        {
            'value': 'Australia'
        }
    ],
    'schema:longitude': [
        {
            'value': '151.209900'
        }
    ],
    'schema:latitude': [
        {
            'value': '-33.865143'
        }
    ],
    
}

payload = omeka_auth.prepare_item_payload(test_place)
new_place = omeka_auth.add_item(payload)

Now we can just call `OmekaAPIClient.add_marker_to_item` without any coordinates, and it will get them from `schema:longitude` and `schema:latitude`.

In [ ]:
new_marker = omeka_auth.add_marker_to_item(new_place['o:id'])

assert new_marker['@type'] == 'o-module-mapping:Marker'
assert new_marker['o:item']['o:id'] == new_place['o:id']
assert new_marker['o-module-mapping:lat'] == -33.865143

## Managing templates

Resource templates can be exported from one Omeka instance and imported into another. This is a really useful way of sharing and resuing data structures. However, exported templates can't be uploaded via the API without some modification, as they don't include Omeka numeric identifiers for referenced classes and properties. Before uploading them, we need to find the referenced classes and properties in the local Omeka instance, and insert the local identifiers into the payload.

Similarly, if custom vocabs are referenced in the exported template, they will need to be updated to include the vocab's identifier in the new Omeka instance. Of course, you'll need to make sure it's installed first.

In general, you want to make sure that the any vocabs and data types referenced by the exported templates will be available in the new Omeka instance. So before you upload any templates you should check that:

* referenced vocabularies are already installed
* modules used in defining data types are installed ( eg the [Numeric Data Types](https://omeka.org/s/modules/NumericDataTypes/) module provides additional data types such as `Timestamp` for dates
* any custom vocabularies (created using the [Custom Vocab](https://omeka.org/s/modules/CustomVocab/) are installed, along with the module itself of course
* the [Value Suggest](https://omeka.org/s/modules/ValueSuggest/) module is installed if the template includes references to the controlled vocabs it provides

In [ ]:
show_doc(OmekaAPIClient.prepare_template_payload)

<h4 id="OmekaAPIClient.prepare_template_payload" class="doc_header"><code>OmekaAPIClient.prepare_template_payload</code><a href="__main__.py#L583" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.prepare_template_payload</code>(**`template_file`**)

Insert local property, class, and vocab identifiers into a resource template
exported from Omeka so that it can be uploaded to the local instance.

Parameters:
* `template_file` - path to a template exported from Omeka (str or pathlib Path)

Returns:
* template payload with local identifiers inserted

In [ ]:
omeka_auth = OmekaAPIClient(
    api_url='https://timsherratt.org/collections/api',
    key_identity=os.getenv('KEY_IDENTITY'), 
    key_credential=os.getenv('KEY_CREDENTIAL')
)

In [ ]:
template_payload = omeka_auth.prepare_template_payload('templates/Person.json')

# Check the label
assert template_payload['o:label'] == 'Person'

# Check that there is a list of properties
assert len(template_payload['o:resource_template_property']) > 0

In [ ]:
show_doc(OmekaAPIClient.upload_template)

<h4 id="OmekaAPIClient.upload_template" class="doc_header"><code>OmekaAPIClient.upload_template</code><a href="__main__.py#L636" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.upload_template</code>(**`template_payload`**)

Upload a template exported from an instance of Omeka to the current local instance.

Parameters:
* `template_payload` - dict payload generated by `prepare_template_payload`

Return:
* dict containing a JSON-LD representation of the uploaded template

In [ ]:
# First we'll check to see if there's already a template with the label 'Person'
existing_template = omeka_auth.get_template_by_label('Person')

# If there is we'll delete it
# Trying to upload a template with the same label as an existing template will generate an error
if existing_template:
    deleted = omeka_auth.delete_resource(existing_template['o:id'], 'resource_templates')

# Now we can add the new template
template_payload = omeka_auth.prepare_template_payload('templates/Person.json')
new_template = omeka_auth.upload_template(template_payload)

# Result should be a 'ResourceTemplate
assert new_template['@type'] == 'o:ResourceTemplate'

# Label should be 'Person'
assert new_template['o:label'] == 'Person'

# It should have properties
assert len(new_template['o:resource_template_property']) > 0

In [ ]:
show_doc(OmekaAPIClient.localise_custom_vocabs)

<h4 id="OmekaAPIClient.localise_custom_vocabs" class="doc_header"><code>OmekaAPIClient.localise_custom_vocabs</code><a href="__main__.py#L517" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.localise_custom_vocabs</code>(**`data_types`**)

Check a list of data types for references to custom vocabs.
If found, look for the local identifier of the custom vocab, 
and insert it into the data type information.

Parameters:
* `data_types` - a list of data types from an exported template property

Returns:
* list of datatypes with local identifiers

In [ ]:
# Data type example with ridiculously high customvocab id value
data_types = [{'name': 'customvocab:200000', 'label': "Date qualifiers"}]

# Localise!
localised_data_types = omeka_auth.localise_custom_vocabs(data_types)
print(localised_data_types)

# Localised data type should start with 'customvocab'
assert localised_data_types[0].startswith('customvocab:')

# The localised customvocab id should not be the same as the supplied one
assert localised_data_types[0] != data_types[0]['name']

['customvocab:1']


In [ ]:
show_doc(OmekaAPIClient.get_template_class_id)

<h4 id="OmekaAPIClient.get_template_class_id" class="doc_header"><code>OmekaAPIClient.get_template_class_id</code><a href="__main__.py#L542" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.get_template_class_id</code>(**`template`**)

Get the local id of the resource class associated with the supplied template.

Parameters:
* `template` - dict from exported template

Returns:
* class identifier

In [ ]:
# Load template from filesystem
template = json.loads(Path('templates/Person.json').read_bytes())

person_class_id = omeka_auth.get_resource_by_term('schema:Person', 'resource_classes')['o:id']

# Get resource id
class_id = omeka_auth.get_template_class_id(template)

assert class_id == person_class_id

In [ ]:
show_doc(OmekaAPIClient.get_template_property_id)

<h4 id="OmekaAPIClient.get_template_property_id" class="doc_header"><code>OmekaAPIClient.get_template_property_id</code><a href="__main__.py#L562" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.get_template_property_id</code>(**`template`**, **`term`**)

Get the local id of the property associated with the supplied template.

Parameters:
* `template` - dict from exported template
* `term` - property term (eg 'o:title_property')

Returns:
* property identifier

In [ ]:
# Load template from filesystem
template = json.loads(Path('templates/Person.json').read_bytes())

# get the id of the schema:name property
name_prop_id = omeka_auth.get_resource_by_term('schema:name', 'properties')['o:id']

# Get id of template property used for title
prop_id = omeka_auth.get_template_property_id(template, 'o:title_property')

# The ids should be the same
assert prop_id == name_prop_id

## Utilities

In [ ]:
show_doc(OmekaAPIClient.format_resource_id)

<h4 id="OmekaAPIClient.format_resource_id" class="doc_header"><code>OmekaAPIClient.format_resource_id</code><a href="__main__.py#L46" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.format_resource_id</code>(**`resource_id`**, **`resource_type`**)

Generate a formatted id for the resource with the specified Omeka id number and resource type.

Parameters:
* `resource_id` - numeric identifier used by Omeka for this resource
* `resource_type` - one of Omeka's resource types, eg: 'items', 'properties'

Returns:
* a dict with values for '@id' and 'o:id'

In [ ]:
id_url = omeka.format_resource_id(5, 'resource_templates')

assert id_url == {
    '@id': 'http://timsherratt.org/collections/api/resource_templates/5',
    'o:id': 5
}

In [ ]:
show_doc(OmekaAPIClient.prepare_property_value)

<h4 id="OmekaAPIClient.prepare_property_value" class="doc_header"><code>OmekaAPIClient.prepare_property_value</code><a href="__main__.py#L266" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.prepare_property_value</code>(**`value`**, **`property_id`**)

Formats a property value according to its datatype as expected by Omeka. 
The formatted value can be used in a payload to create a new item.

Parameters:
* `value` - a dict containing a `value` and (optionally) a `type`
* `property_id` - the numeric identifier of the property

Note that is no `type` is supplied, 'literal' will be used by default.

Returns:
* a dict with values for `property_id`, `type`, and either `@id` or `@value`.

In [ ]:
prop_id = omeka_auth.get_property_id('schema:url')
prop_value = {'value': 'https://glam-workbench.net', 'type': 'uri'}

formatted_prop = omeka_auth.prepare_property_value(prop_value, prop_id)
formatted_prop

{'property_id': 393, 'type': 'uri', '@id': 'https://glam-workbench.net'}

In [ ]:
assert prop_id == formatted_prop['property_id']
assert 'type' in formatted_prop
assert formatted_prop['@id'] == 'https://glam-workbench.net'

In [ ]:
show_doc(OmekaAPIClient.get_template_properties)

<h4 id="OmekaAPIClient.get_template_properties" class="doc_header"><code>OmekaAPIClient.get_template_properties</code><a href="__main__.py#L238" class="source_link" style="float:right">[source]</a></h4>

> <code>OmekaAPIClient.get_template_properties</code>(**`template_id`**)

List properties used by the specified template.

The resource template objects returned by the API don't include property terms.
This function gets the additional details, and organises the properties in a dictionary, 
organised by term. This makes it easy to check if a particular term is used by a template.

Parameters:
* `template_id` - numeric identifier for a template

Returns:
* a dict organised by property terms, with values for `property_id` and `type`

In [ ]:
template_id = omeka_auth.get_template_by_label('Newspaper')['o:id']
newspaper_properties = omeka_auth.get_template_properties(template_id)

assert 'schema:name' in newspaper_properties

## Example: adding a newspaper article from Trove

In the example below I'm going to manually step through the process of adding a new item to Omeka using the API in order to demonstrate the methods available. But of course the point of using the API is to automate such processes -- joining together the individual steps so they can be embedded into your own systems or workflows. For a more detailed example that uploads Trove newspaper articles from a variety of sources, including Trove searches, Trove lists, and Zotero libraries, see the [GLAM Workbench](https://glam-workbench.net/trove-newspapers/#upload-trove-newspaper-articles-to-omeka-s).

Let's suppose we want to add [this newspaper article](http://nla.gov.au/nla.news-article226799674) in Trove to our Omeka instance. To take best advantage of Omeka's linked data infrastructure, we'll actually create two resources -- one for the article, and one for the newspaper it was published in.

I've already created templates labelled 'Newspaper' and 'Newspaper article'.

Let's start with the newspaper. First we need to find out the numeric identifier Omeka is using for the Newspaper template. We can use `OmekaAPIClient.get_template_by_label` to find out. 

In [ ]:
omeka_auth = OmekaAPIClient(
    api_url='http://timsherratt.org/collections/api',
    key_identity=os.getenv('KEY_IDENTITY'), 
    key_credential=os.getenv('KEY_CREDENTIAL')
)

newspaper_template_id = omeka_auth.get_template_by_label('Newspaper')['o:id']
newspaper_template_id

5

For convenience, we can get a summary of the properties used in the Newspaper template using `OmekaAPIClient.get_template_properties`. This is useful if we want to check which properties are in use, and what data types are expected.

In [ ]:
omeka_auth.get_template_properties(newspaper_template_id)

{'schema:name': {'property_id': 1116, 'type': ['literal']},
 'schema:additionalType': {'property_id': 1199, 'type': ['uri']},
 'schema:description': {'property_id': 528,
  'type': ['literal', 'uri', 'resource:item']},
 'schema:alternateName': {'property_id': 282,
  'type': ['literal', 'uri', 'resource:item']},
 'schema:editor': {'property_id': 752, 'type': ['resource:item']},
 'schema:publisher': {'property_id': 967, 'type': ['resource:item']},
 'schema:issn': {'property_id': 931,
  'type': ['literal', 'uri', 'resource:item']},
 'schema:startDate': {'property_id': 513, 'type': ['numeric:timestamp']},
 'schema:endDate': {'property_id': 657, 'type': ['numeric:timestamp']},
 'schema:locationCreated': {'property_id': 706, 'type': ['resource:item']},
 'schema:hasPart': {'property_id': 1405, 'type': ['resource:item']},
 'schema:about': {'property_id': 1392, 'type': ['resource:item']},
 'schema:mentions': {'property_id': 747, 'type': ['resource:item']},
 'schema:contentLocation': {'property_i

The first step in preparing our data for upload is to create a dictionary, using the required property terms as keys. In this case, we'll assign data about the newspaper to `schema.name` and `schema.url`.

In [ ]:
newspaper = {
    'schema.name': [
        {
            'type': 'literal',
            'value': 'The Bendigo Independent (Vic. : 1891 - 1918)'
        }
    ],
    'schema:url': [
        {
            'type': 'literal',
            'value': 'http://nla.gov.au/nla.news-title806'
        }
    ]
}

Now we can use this data to create a payload for upload to Omeka. `OmekaAPIClient.prepare_item_payload_using_template` will check our data against the Newspaper template and build the payload.

In [ ]:
payload = omeka_auth.prepare_item_payload_using_template(newspaper, newspaper_template_id)

Term schema.name not in template
Data type "literal" for term "schema:url" not allowed by template


Whoops! It seems we have some problems with our data file! First of all we've used a full stop, rather than a colon in `schema:name`. Second, we've said the data type of `schema:url` is 'literal', but if we check the template properties we'll see it' should be 'uri'. Let's make these changes.

In [ ]:
newspaper = {
    'schema:name': [
        {
            'type': 'literal',
            'value': 'The Bendigo Independent (Vic. : 1891 - 1918)'
        }
    ],
    'schema:url': [
        {
            'type': 'uri',
            'value': 'http://nla.gov.au/nla.news-title806'
        }
    ]
}

In [ ]:
payload = omeka_auth.prepare_item_payload_using_template(newspaper, newspaper_template_id)

That seems better! Now we can examine the payload.

In [ ]:
payload

{'schema:name': [{'property_id': 1116,
   'type': 'literal',
   '@value': 'The Bendigo Independent (Vic. : 1891 - 1918)'}],
 'schema:url': [{'property_id': 393,
   'type': 'uri',
   '@id': 'http://nla.gov.au/nla.news-title806'}]}

Notice how the values have been reformatted? They should now be ready to upload to Omeka using `OmekaAPIClient.add_item`. We'll supply both the payload and the newspaper template id.

In [ ]:
newspaper_item = omeka_auth.add_item(payload, template_id=newspaper_template_id)

We can check the contents of `newspaper_item` to make sure it's been added succesfully.

In [ ]:
newspaper_item

{'@context': 'http://timsherratt.org/collections/api-context',
 '@id': 'http://timsherratt.org/collections/api/items/1045',
 '@type': ['o:Item', 'schema:Newspaper'],
 'o:id': 1045,
 'o:is_public': True,
 'o:owner': {'@id': 'http://timsherratt.org/collections/api/users/1',
  'o:id': 1},
 'o:resource_class': {'@id': 'http://timsherratt.org/collections/api/resource_classes/161',
  'o:id': 161},
 'o:resource_template': {'@id': 'http://timsherratt.org/collections/api/resource_templates/5',
  'o:id': 5},
 'o:thumbnail': None,
 'o:title': 'The Bendigo Independent (Vic. : 1891 - 1918)',
 'thumbnail_display_urls': {'large': None, 'medium': None, 'square': None},
 'o:created': {'@value': '2022-04-04T00:48:33+00:00',
  '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
 'o:modified': {'@value': '2022-04-04T00:48:33+00:00',
  '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
 'o:media': [],
 'o:item_set': [],
 'o:site': [],
 'schema:name': [{'type': 'literal',
   'property_id': 1116,
   '

Great! Now what about the article? Again let's start by having a look at the 'Newspaper article' template.

In [ ]:
article_template_id = omeka_auth.get_template_by_label('Newspaper article')['o:id']
omeka_auth.get_template_properties(article_template_id)

{'schema:name': {'property_id': 1116,
  'type': ['literal', 'uri', 'resource:item']},
 'schema:additionalType': {'property_id': 1199, 'type': ['uri']},
 'schema:description': {'property_id': 528,
  'type': ['literal', 'uri', 'resource:item']},
 'schema:alternateName': {'property_id': 282,
  'type': ['literal', 'uri', 'resource:item']},
 'schema:datePublished': {'property_id': 928, 'type': ['numeric:timestamp']},
 'schema:pagination': {'property_id': 376,
  'type': ['literal', 'uri', 'resource:item']},
 'schema:creator': {'property_id': 329, 'type': ['resource:item']},
 'schema:contributor': {'property_id': 1303, 'type': ['resource:item']},
 'schema:isPartOf': {'property_id': 736, 'type': ['resource:item']},
 'schema:license': {'property_id': 1348,
  'type': ['literal', 'uri', 'resource:item']},
 'schema:about': {'property_id': 1392, 'type': ['resource:item']},
 'schema:mentions': {'property_id': 747, 'type': ['resource:item']},
 'schema:contentLocation': {'property_id': 1419, 'type': [

This time we'll build our data file in stages.

In [ ]:
# Create an empty dictionary
article = {}

# Add value for schema:name
article['schema:name'] = ["MR WRAGGE'S PREDICTION. RENEWAL OF CYCLONE FORETOLD."]

You'll notice that this time I've only included the value of the property, not the data type. This is because `OmekaAPIClient.prepare_item_payload_using_template` will insert default data types if they're not specified. The basic rules it uses if you don't supply a data type are:

* if the template defines a default data type, use that
* if 'literal' is in the template's list of expected data types, use that
* if 'literal' is not in the template's list of expected data types print a warning asking the user to specify a data type and drop the property from the payload

In the case above, the 'literal' data type will be assigned to the `schema:name` value.

To create a link between the article and the newspaper it was published in, we can use `schema.isPartOf`. If we look in the list of template properties we see that this property expects a data type of 'resource:item'.

```
'schema:isPartOf': {'property_id': 736, 'type': ['resource:item']}
```

The 'resource:item' data type means that the value will be an identifier pointing to another Omeka item. So we include the identifier of the newly-created newspaper record. Once again, the data type will be automatically added when we generate the payload.

In [ ]:
article['schema:isPartOf'] = [newspaper_item['o:id']]

Next we'll add the publication date. To recognise dates as specific data types, you need to install the [Numeric Data Types](https://omeka.org/s/modules/NumericDataTypes/) module. Once that's done, you can set the 'numeric:timestamp' data type as the default for any date fields in your template. In the list of properties from the newspaper article template, you can see:

```
'schema:datePublished': {'property_id': 928, 'type': ['numeric:timestamp']}
```

The date value itself is supplied in a standard ISO format -- 'YYYY-MM-DD'.

In [ ]:
article['schema:datePublished'] = ['1918-03-14']

Finally we'll add a link to the article. Once again the template includes a default data type.

```
'schema:url': {'property_id': 393, 'type': ['uri']}
```

So we can just include the value.

In [ ]:
article['schema:url'] = ['http://nla.gov.au/nla.news-article226799674']

Now that we've assembled the article metadata, we can generate a payload.

In [ ]:
article_payload = omeka_auth.prepare_item_payload_using_template(article, article_template_id)

As you can see below, the default data types have been inserted into the payload, and some additional fields have been added to define the link to the newspaper item.

In [ ]:
article_payload

{'schema:name': [{'property_id': 1116,
   'type': 'literal',
   '@value': "MR WRAGGE'S PREDICTION. RENEWAL OF CYCLONE FORETOLD."}],
 'schema:isPartOf': [{'property_id': 736,
   'type': 'resource:item',
   '@id': 'http://timsherratt.org/collections/api/items/1045',
   'value_resource_id': 1045,
   'value_resource_name': 'items'}],
 'schema:datePublished': [{'property_id': 928,
   'type': 'numeric:timestamp',
   '@value': '1918-03-14'}],
 'schema:url': [{'property_id': 393,
   'type': 'uri',
   '@id': 'http://nla.gov.au/nla.news-article226799674'}]}

We could now go ahead and upload the payload to Omeka, but what if we want to include media files? In this case I have a JPG image of the article I want to attach. All you need to do is supply a list of paths pointing to media files using the `media_files` parameter. The list can just include the paths as strings, in which case the file name will be used as the media object's title. Alternatively, you can supply a list of dicts, each containing a `path` and `title` value.

In [ ]:
# Create a list of paths pointing to media files
# The paths can be strings or pathlib Paths
media_files = ['media/nla.news-article226799674-24144902.jpg']

# Include the media files when we upload the payload
article_item = omeka_auth.add_item(article_payload, media_files=media_files, template_id=article_template_id)

When we look at the uploaded item, we'll see that the media files have been processed and added to the record.

In [ ]:
article_item

{'@context': 'http://timsherratt.org/collections/api-context',
 '@id': 'http://timsherratt.org/collections/api/items/1046',
 '@type': ['o:Item', 'schema:NewsArticle'],
 'o:id': 1046,
 'o:is_public': True,
 'o:owner': {'@id': 'http://timsherratt.org/collections/api/users/1',
  'o:id': 1},
 'o:resource_class': {'@id': 'http://timsherratt.org/collections/api/resource_classes/543',
  'o:id': 543},
 'o:resource_template': {'@id': 'http://timsherratt.org/collections/api/resource_templates/4',
  'o:id': 4},
 'o:thumbnail': None,
 'o:title': "MR WRAGGE'S PREDICTION. RENEWAL OF CYCLONE FORETOLD.",
 'thumbnail_display_urls': {'large': 'http://timsherratt.org/collections/files/large/c60918ee88cb729ae8803742bbbb185460700903.jpg',
  'medium': 'http://timsherratt.org/collections/files/medium/c60918ee88cb729ae8803742bbbb185460700903.jpg',
  'square': 'http://timsherratt.org/collections/files/square/c60918ee88cb729ae8803742bbbb185460700903.jpg'},
 'o:created': {'@value': '2022-04-04T00:48:40+00:00',
 